# Case Study: Algorithmic Trading Analysis of Historical Cryptocurrency Data for Bitcoin Price Prediction using Keras


## About Dataset

### Context

This is historical data on cryptocurrency tradings for the period from 2016-01-01 to 2022-01-08.
We are going to focus only on the Bitcoin cryptocurrency because of the luck of time , the original dataset has over 2.3 million rows.

### IMPORTING DATA

In [10]:
import numpy as np 
import pandas as pd 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,AveragePooling1D,Reshape
from sklearn.metrics import mean_absolute_error

2023-12-13 15:34:06.421173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("/Users/mobilegalery/Downloads/crypto_tradinds.csv")
df

/var/folders/2p/03hj35nj3n3c4mp7p1dvdcm80000gn/T/ipykernel_99948/1797222102.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mobilegalery/Downloads/crypto_tradinds.csv")


,trade_date,volume,price_usd,price_btc,market_cap,capitalization_change_1_day,USD_price_change_1_day,BTC_price_change_1_day,crypto_name,crypto_type,ticker,max_supply,site_url,github_url,minable,platform_name,industry_name
0,2016-01-01,36278900.00,434.330000,1.000000,6.529300e+09,0.000000,0.000000,0.000000,Bitcoin,0.0,BTC,2.100000e+07,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
1,2016-01-02,30096600.00,433.440000,1.000000,6.517390e+09,-0.001824,-0.002049,0.000000,Bitcoin,0.0,BTC,2.100000e+07,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
2,2016-01-03,39633800.00,430.010000,1.000000,6.467430e+09,-0.007666,-0.007913,0.000000,Bitcoin,0.0,BTC,2.100000e+07,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
3,2016-01-04,38477500.00,433.090000,1.000000,6.515713e+09,0.007466,0.007163,0.000000,Bitcoin,0.0,BTC,2.100000e+07,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
4,2016-01-05,34522600.00,431.960000,1.000000,6.500393e+09,-0.002351,-0.002609,0.000000,Bitcoin,0.0,BTC,2.100000e+07,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382638,2020-10-31,39594.53,0.040280,0.000003,0.000000e+00,0.000000,0.013078,-0.004158,PlotX,1.0,PLOT,2.000000e+08,https://plotx.io/,https://github.com/plotx/smart-contracts,0.0,NaN,NaN
2382639,2020-11-01,15120.45,0.041831,0.000003,0.000000e+00,0.000000,0.038505,0.041823,PlotX,1.0,PLOT,2.000000e+08,https://plotx.io/,https://github.com/plotx/smart-contracts,0.0,NaN,NaN
2382640,2020-10-30,14383.05,1.580000,0.000117,0.000000e+00,0.000000,-0.076023,-0.083433,SableAssent,1.0,SAC1,6.000000e+09,https://www.sableassent.com,https://attachments.office.net/owa/daryl%40sab...,0.0,NaN,NaN
2382641,2020-10-31,3720.08,1.500000,0.000109,0.000000e+00,0.000000,-0.050633,-0.066785,SableAssent,1.0,SAC1,6.000000e+09,https://www.sableassent.com,https://attachments.office.net/owa/daryl%40sab...,0.0,NaN,NaN


In [3]:
df.columns

Index(['trade_date', 'volume', 'price_usd', 'price_btc', 'market_cap',
       'capitalization_change_1_day', 'USD_price_change_1_day',
       'BTC_price_change_1_day', 'crypto_name', 'crypto_type', 'ticker',
       'max_supply', 'site_url', 'github_url', 'minable', 'platform_name',
       'industry_name'],
      dtype='object')

In [5]:
#Select Bitcoin Data
btc_data = df[df['ticker']=='BTC']
btc_data

,trade_date,volume,price_usd,price_btc,market_cap,capitalization_change_1_day,USD_price_change_1_day,BTC_price_change_1_day,crypto_name,crypto_type,ticker,max_supply,site_url,github_url,minable,platform_name,industry_name
0,2016-01-01,3.627890e+07,434.33,1.0,6.529300e+09,0.000000,0.000000,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
1,2016-01-02,3.009660e+07,433.44,1.0,6.517390e+09,-0.001824,-0.002049,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
2,2016-01-03,3.963380e+07,430.01,1.0,6.467430e+09,-0.007666,-0.007913,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
3,2016-01-04,3.847750e+07,433.09,1.0,6.515713e+09,0.007466,0.007163,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
4,2016-01-05,3.452260e+07,431.96,1.0,6.500393e+09,-0.002351,-0.002609,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,2020-10-29,5.649950e+10,13437.88,1.0,2.489953e+11,0.012588,0.012553,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
1742,2020-10-30,3.058149e+10,13546.52,1.0,2.510182e+11,0.008124,0.008085,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
1743,2020-10-31,3.030646e+10,13780.99,1.0,2.553721e+11,0.017345,0.017309,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)
1744,2020-11-01,2.445386e+10,13737.11,1.0,2.545698e+11,-0.003142,-0.003184,0.0,Bitcoin,0.0,BTC,21000000.0,https://bitcoin.org/,https://github.com/bitcoin/,1.0,XRP,Proof of Work (PoW)


In [6]:
btc_data.nunique()

trade_date                     1746
volume                         1746
price_usd                      1740
price_btc                         2
market_cap                     1746
capitalization_change_1_day    1746
USD_price_change_1_day         1744
BTC_price_change_1_day            1
crypto_name                       1
crypto_type                       1
ticker                            1
max_supply                        1
site_url                          1
github_url                        1
minable                           1
platform_name                     1
industry_name                     1
dtype: int64

**Let's drop the columns with less than 2 unique values because their are not useful**

In [7]:
drop_columns_list = btc_data.nunique()[btc_data.nunique()<=2].index
btc_data.drop(drop_columns_list, axis=1, inplace=True)
btc_data

/var/folders/2p/03hj35nj3n3c4mp7p1dvdcm80000gn/T/ipykernel_99948/1194763639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_data.drop(drop_columns_list, axis=1, inplace=True)


,trade_date,volume,price_usd,market_cap,capitalization_change_1_day,USD_price_change_1_day
0,2016-01-01,3.627890e+07,434.33,6.529300e+09,0.000000,0.000000
1,2016-01-02,3.009660e+07,433.44,6.517390e+09,-0.001824,-0.002049
2,2016-01-03,3.963380e+07,430.01,6.467430e+09,-0.007666,-0.007913
3,2016-01-04,3.847750e+07,433.09,6.515713e+09,0.007466,0.007163
4,2016-01-05,3.452260e+07,431.96,6.500393e+09,-0.002351,-0.002609
...,...,...,...,...,...,...
1741,2020-10-29,5.649950e+10,13437.88,2.489953e+11,0.012588,0.012553
1742,2020-10-30,3.058149e+10,13546.52,2.510182e+11,0.008124,0.008085
1743,2020-10-31,3.030646e+10,13780.99,2.553721e+11,0.017345,0.017309
1744,2020-11-01,2.445386e+10,13737.11,2.545698e+11,-0.003142,-0.003184


In [8]:
btc_data.columns

Index(['trade_date', 'volume', 'price_usd', 'market_cap',
       'capitalization_change_1_day', 'USD_price_change_1_day'],
      dtype='object')

### Printing full graph of bitcoin price

In [22]:
fig = go.Figure(data=go.Scatter(x=btc_data['trade_date'], y=btc_data['price_usd']))
fig.show()

### Data Preprocessing and Splitting Function

The following function, **data_preproc_and_split**, is designed for the preprocessing and splitting of time-series data suitable for training Long Short-Term Memory (LSTM) models. It takes a dataset (data) and a parameter (n) that represents the timeframe size for each data point.

**Function Overview:**

**Variable Initialization:**

The function initializes empty DataFrames (train, target, pred_convert) to store the training data, target values, and starting prices for conversion.

**Data Preprocessing:**

The function performs preprocessing on the input data by normalizing prices and volumes relative to a reference point (def_nom).
It constructs the training set (train) and target set (target) based on the specified timeframe size (n).
Starting prices for conversion are saved in the pred_convert DataFrame.

**Data Splitting:**

The function splits the processed data into training and validation sets.
It also prepares the target sets (y_train, y_valid) by removing the 'date' column.

**Data Shape Transformation:**

The function reshapes the training and validation sets into a format suitable for LSTM models ((num_samples, n, 2)).

**Parameters:**

**data:** 
The input time-series dataset.
n: Timeframe size for each data point.

**Returns:**
x_train: Processed and shaped training data.
x_valid: Processed and shaped validation data.
y_train: Target values for training.
y_valid: Target values for validation.
target: DataFrame containing target values and dates.
pred_convert: DataFrame containing starting prices for conversion.

In [23]:
def data_preproc_and_split(data,n):
    col = []
    for i in range(n):
        col.append('price' + str(i))
        col.append('volume' + str(i))
    train = pd.DataFrame(columns = col)
    target = pd.DataFrame(columns = ['date','price'])
    pred_convert = pd.DataFrame(columns = ['date','price'])
    
    for i in range(1,len(data)-n-1):
        def_nom = data.loc[i-1, 'price_usd']
        for j in range(n):
            train.loc[i, 'price' + str(j)] = data.loc[i+j, 'price_usd']/def_nom-1
            train.loc[i, 'volume' + str(j)] = data.loc[i+j, 'volume']/data.loc[i+j, 'market_cap']
        target.loc[i, 'price'] = data.loc[i+n+1, 'price_usd']/def_nom-1
        target.loc[i, 'date'] = data.loc[i+n+1, 'trade_date']  
        pred_convert.loc[i, 'price'] = def_nom
        pred_convert.loc[i, 'date'] = data.loc[i+n+1, 'trade_date'] 

    x_train = train.iloc[:train.shape[0]-100]
    x_valid = train.iloc[train.shape[0]-100:]
    y_train = target.iloc[:target.shape[0]-100]
    y_valid = target.iloc[target.shape[0]-100:]
    y_train.drop(['date'], axis=1, inplace=True)
    y_valid.drop(['date'], axis=1, inplace=True)
    
    x_train = x_train.to_numpy().reshape((x_train.shape[0],n,2))
    x_valid = x_valid.to_numpy().reshape((x_valid.shape[0],n,2))
    return x_train,x_valid,y_train,y_valid,target,pred_convert

## Model Generating 

The next function generates and trains an LSTM neural network for time-series prediction.

**Parameters:**

x: Input data shaped as (num_samples, n, 2).
y: Target values for training.
n: Timeframe size.
e: Number of training epochs (default: 75).
v: Verbosity level during training (default: 0).

**Returns:**

Trained LSTM model.

In [24]:
def model_gen(x,y,n,e=75,v=0):
    mod = Sequential()
    mod.add(LSTM(32,return_sequences=True,input_shape=(n,2)))
    mod.add(LSTM(64))
    mod.add(Dropout(0.35))
    mod.add(Dense(128, activation='relu'))
    mod.add(Dense(1))
    mod.compile(optimizer='adam',loss='mse')
    mod.fit(x,y,epochs=e,shuffle=False,verbose=v)
    return mod

## Model training and evaluation

This code perform the data preprocessing and splitting, trains our model with varying epochs, and visualizes the predictions against actual prices. It evaluates model performance using Mean Absolute Error (MAE) before and after converting predicted prices. The results, including MAE values, are printed for different epochs.It aims to find optimal epoch for model

In [17]:
x_train, x_valid, y_train, y_valid, target, pred_convert = data_preproc_and_split(btc_data, 15)

y_pred = {}
mae = {'before_convert': {}, 'after_convert': {}}
R = range(25, 130, 25)

x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')

# Model training
for i in R:
    model = model_gen(x_train, y_train.astype('float32'), 15, i, 0)
    preds = model.predict(x_valid)
    
    y_pred[i] = pd.DataFrame(preds, index=y_valid.index, columns=['price']).copy()
    y_pred[i]['date'] = target['date']
    
    mae['before_convert'][i] = mean_absolute_error(y_valid['price'], y_pred[i]['price'])
    
    y_pred[i]['price'] = pred_convert['price'].astype('float32') * (y_pred[i]['price'] + 1)
    
    mae['after_convert'][i] = mean_absolute_error(btc_data.iloc[btc_data.shape[0] - 100:]['price_usd'], y_pred[i]['price'])

# Visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc_data.iloc[btc_data.shape[0] - 100:]['trade_date'], y=btc_data.iloc[btc_data.shape[0] - 100:]['price_usd'], name='Real price'))

for i in R:
    fig.add_trace(go.Scatter(x=y_pred[i]['date'], y=y_pred[i]['price'], name='Epoch = ' + str(i)))

fig.show()

for i in R:
    print(f'Epoch = {i}. MAE before price conversion: {mae["before_convert"][i]}. MAE after price conversion: {mae["after_convert"][i]}.')


<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4/4 [==============================] - 1s 5ms/step


Epoch = 25. MAE before price conversion: 0.026322311054904995. MAE after price conversion: 280.15764726562503.
Epoch = 50. MAE before price conversion: 0.024027830969089517. MAE after price conversion: 254.90789101562498.
Epoch = 75. MAE before price conversion: 0.02411795018568267. MAE after price conversion: 255.68744609375005.
Epoch = 100. MAE before price conversion: 0.024602129981326982. MAE after price conversion: 260.7005390625.
Epoch = 125. MAE before price conversion: 0.024162679323960698. MAE after price conversion: 256.01084921875.


This code aims to find optimal size of timeframe.

In [20]:
y_pred = {}
mae = {'before_convert': {}, 'after_convert': {}}
R = range(10, 31, 5)

for i in R:
    x_train, x_valid, y_train, y_valid, target, pred_convert = data_preproc_and_split(btc_data, i)
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    y_train = y_train.astype('float32')
    y_valid = y_valid.astype('float32')

    model = model_gen(x_train, y_train, i, 75, 0)  
    preds = model.predict(x_valid)
    y_pred[i] = pd.DataFrame(preds, index=y_valid.index, columns=['price'])
    y_pred[i]['date'] = target['date']
    mae['before_convert'][i] = mean_absolute_error(y_valid['price'], y_pred[i]['price'])
    y_pred[i]['price'] = pred_convert['price'] * (y_pred[i]['price'] + 1)
    mae['after_convert'][i] = mean_absolute_error(btc_data.iloc[btc_data.shape[0] - 100:]['price_usd'], y_pred[i]['price'])

# Visualization 
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc_data.iloc[btc_data.shape[0] - 100:]['trade_date'], y=btc_data.iloc[btc_data.shape[0] - 100:]['price_usd'], name='Real price'))

for i in R:
    fig.add_trace(go.Scatter(x=y_pred[i]['date'], y=y_pred[i]['price'], name='TimeFrame Size = ' + str(i)))

fig.show()

for i in R:
    print(f'TimeFrame Size = {i}. MAE before price conversion: {mae["before_convert"][i]}. MAE after price conversion: {mae["after_convert"][i]}.')


<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4/4 [==============================] - 1s 4ms/step


<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4/4 [==============================] - 1s 5ms/step


<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4/4 [==============================] - 1s 6ms/step


<timed exec>:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<timed exec>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



4/4 [==============================] - 1s 9ms/step


TimeFrame Size = 10. MAE before price conversion: 0.02415022812783718. MAE after price conversion: 258.3171287709534.
TimeFrame Size = 15. MAE before price conversion: 0.024647634476423264. MAE after price conversion: 261.32179457930914.
TimeFrame Size = 20. MAE before price conversion: 0.025219451636075974. MAE after price conversion: 263.75228928647647.
TimeFrame Size = 25. MAE before price conversion: 0.02740953490138054. MAE after price conversion: 284.36052811353807.
TimeFrame Size = 30. MAE before price conversion: 0.02589448168873787. MAE after price conversion: 266.3189007321357.


In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc_data.iloc[btc_data.shape[0]-100:]['trade_date'], y=btc_data.iloc[btc_data.shape[0]-100:]['price_usd'], name='Real price'))
fig.add_trace(go.Scatter(x=y_pred[15]['date'], y=y_pred[15]['price'], name='Predict price'))
fig.show()

**NOTE:** 

I have selected this mini-project to demonstrate my enthusiasm for trading algorithms. I hope my work aligns with your expectations, and I am eager to delve deeper into this field. 
I am committed to learning more and practicing further with your guidance. Thank you for your attention and support!